In [1]:
import pandas as pd
import numpy as np

In [2]:
# Load the CSV file
lists = pd.read_csv("csvs/lists_2019_8.csv",encoding="ISO-8859-1")
commits = pd.read_csv("csvs/modified_commits_2019_8.csv",encoding="ISO-8859-1")
people = pd.read_csv("csvs/people_2019_8.csv",encoding="ISO-8859-1")
pr = pd.read_csv("csvs/pull-requests.csv",encoding="ISO-8859-1")
alias=pd.read_csv("csvs/modified_aliases_2019_8.csv",encoding="ISO-8859-1")

/tmp/ipykernel_261844/1690718824.py:3: DtypeWarning: Columns (14) have mixed types. Specify dtype option on import or set low_memory=False.
  commits = pd.read_csv("csvs/modified_commits_2019_8.csv",encoding="ISO-8859-1")


In [3]:
lists.columns

Index(['listid', 'listname', 'pj_alias', 'status', 'start_date', 'end_date',
       'dev_is_available', 'commit_is_available', 'is_in_attic', 'sponsor',
       'intro', 'istoplv', 'start_from_scratch', 'pj_url', 'pj_github_url'],
      dtype='object')

In [25]:
a=lists[['listid','listname']]
a['listname'] = a['listname'].str.lower()  # Convert to lowercase
a = a.sort_values(by='listname', ascending=True)  # Sort in ascending order
a.to_csv("project_name.csv",index=False)

/tmp/ipykernel_261844/3454612999.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  a['listname'] = a['listname'].str.lower()  # Convert to lowercase


In [23]:
# Remove "incubator-" from the start of strings
pr["repo"] = pr["repo"].str.replace(r"^incubator-", "", regex=True)
pr["repo"] = pr["repo"].str.replace(r"^retired-", "", regex=True)

In [24]:
b = pr['repo'].unique()  # Get unique values from the 'repo' column (returns a NumPy array)
b = pd.DataFrame(b, columns=['repo'])  # Convert to DataFrame
b['repo'] = b['repo'].str.lower()  # Convert to lowercase
b = b.sort_values(by='repo', ascending=True)  # Sort in ascending order
b.to_csv("pr.csv",index=False)

In [20]:
df_combined = pd.concat([a, b], axis=1)
df_combined.to_csv("combined.csv",index=False)

In [26]:
# Function to find matches
def find_matching_repo(listname, repo_list):
    for repo in repo_list:
        if pd.notna(listname) and pd.notna(repo):
            if pd.Series([listname]).str.contains(repo, regex=True, case=False).any():
                return repo  # Return first matching repo
    return np.nan  # If no match found

# Apply the function to each row in 'a'
a['matched_repo'] = a['listname'].apply(lambda x: find_matching_repo(x, b['repo'].tolist()))

a.to_csv("combined.csv",index=False)

In [27]:
unique_count = a['matched_repo'].nunique()
print(f"Number of unique values: {unique_count}")


Number of unique values: 220


In [73]:
c=pd.read_csv("combined.csv",encoding="ISO-8859-1")
nan_count = c['matched_repo'].isna().sum()
print(f"Number of NaNs in column : {nan_count}")


Number of NaNs in column : 60


In [37]:
d=pd.read_csv("pr.csv",encoding="ISO-8859-1")

In [39]:
# Find missing values
missing_repos = d[~d['repo'].isin(c['matched_repo'])]  # Select rows where repo is not in listname

# Print missing repos
if not missing_repos.empty:
    print("These repos were not found in listname column of c:")
    print(missing_repos['repo'].tolist())
else:
    print("All repos in d['repo'] are found in c['listname'].")

These repos were not found in listname column of c:
['bval', 'creadur-rat', 'directory-server', 'geronimo-yoko', 'ponymail']


In [71]:
row_261 = lists.loc[lists['pj_alias'] == "woden"]
row_179 = lists.loc[a['listid'] == 179]

In [72]:
# Print the result
row_261

,listid,listname,pj_alias,status,start_date,end_date,dev_is_available,commit_is_available,is_in_attic,sponsor,intro,istoplv,start_from_scratch,pj_url,pj_github_url
249,249,Woden,woden,1,3/24/2005,12/8/2007,t,f,f,Web Services\n\t\t \n\t,Implementation of WSDL 2.0\n\t\t \n\t,NaN,NaN,http://incubator.apache.org/projects/woden.html,https://github.com/apache/Woden


In [46]:
row_179

,listid,listname,pj_alias,status,start_date,end_date,dev_is_available,commit_is_available,is_in_attic,sponsor,intro,istoplv,start_from_scratch,pj_url,pj_github_url
179,179,Open Climate Workbench,climate,1,2/15/2013,2/19/2014,t,t,f,Incubator\n\t\t\n\t\t(Chris A. Mattmann)\n\t\t...,A tool for scalable comparison of remote sensi...,NaN,NaN,http://incubator.apache.org/projects/climate.html,https://github.com/apache/Open Climate Workbench


In [70]:
# Drop NaN values from the column before checking for duplicates
duplicates = c.dropna(subset=['matched_repo'])[c['matched_repo'].duplicated(keep=False)]

# Print the duplicates
if not duplicates.empty:
    print("Duplicate rows (without NaN values):")
    print(duplicates)
else:
    print("No duplicates found in column (after dropping NaN).")



Duplicate rows (without NaN values):
     listid listname matched_repo
184     172      ode          ode
302     249    woden          ode


/tmp/ipykernel_261844/3131390917.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  duplicates = c.dropna(subset=['matched_repo'])[c['matched_repo'].duplicated(keep=False)]


In [76]:
# Convert 'repo' column in pr to lowercase before matching
pr['repo'] = pr['repo'].str.lower()

# Assuming 'pr' and 'c' are your dataframes
pr['project_id'] = pr['repo'].apply(
    lambda repo: c.loc[c['matched_repo'] == repo, 'listid'].values[0] if not c.loc[c['matched_repo'] == repo, 'listid'].empty else np.nan
)

In [77]:
pr

,repo,number,user,user_type,state,created_at,updated_at,closed_at,merged_at,merge_commit_sha,issue_url,body,project_id
0,abdera,4,dargooo,User,closed,2020-11-26T21:21:33Z,2023-05-21T04:37:38Z,2023-05-21T04:37:38Z,NaN,85551716e4900cb514c5913086cedb869edb5ecf,https://api.github.com/repos/apache/abdera/iss...,NaN,49
1,abdera,3,YYTVicky,User,open,2020-03-01T00:37:56Z,2020-04-30T02:01:45Z,NaN,NaN,10b439abf352089d813cf882e9d6291fb58bd363,https://api.github.com/repos/apache/abdera/iss...,NaN,49
2,abdera,2,nicolasyanncouturier,User,open,2013-10-18T13:32:37Z,2014-07-16T01:44:57Z,NaN,NaN,e973e7069a3e4be285b2b934d57adf32caa0a9ef,https://api.github.com/repos/apache/abdera/iss...,Used the same system in BaseSerializer.writeEx...,49
3,abdera,1,nicolasyanncouturier,User,open,2013-10-14T12:23:03Z,2014-07-13T19:17:32Z,NaN,NaN,975791491d738b7d0467bd1975e799857a1c43ed,https://api.github.com/repos/apache/abdera/iss...,The test would most likely need to be split in...,49
4,accumulo,4027,dlmarion,User,closed,2023-12-06T14:20:37Z,2024-07-12T00:15:54Z,2023-12-06T14:34:06Z,2023-12-06T14:34:06Z,29c307fca48117de1433754ead350e031c4f4153,https://api.github.com/repos/apache/accumulo/i...,NaN,50
...,...,...,...,...,...,...,...,...,...,...,...,...,...
569443,zipkin,2379,shakuzen,User,closed,2019-02-01T09:58:52Z,2019-02-04T03:34:15Z,2019-02-02T09:01:02Z,2019-02-02T09:01:01Z,1b2dbdcc50d35dd6f8a5b740a12983ec12d41606,https://api.github.com/repos/openzipkin/zipkin...,RabbitMQ may be managed by a team that will cr...,312
569444,zipkin,2378,codefromthecrypt,User,closed,2019-02-01T09:12:47Z,2019-02-18T06:44:53Z,2019-02-18T06:44:51Z,NaN,32156d935f6be5ebf6da92e3fa6497004d679efa,https://api.github.com/repos/openzipkin/zipkin...,Lookback zero is invalid and should return emp...,312
569445,zipkin,2377,codefromthecrypt,User,closed,2019-02-01T08:55:20Z,2019-02-01T09:51:36Z,2019-02-01T09:51:34Z,2019-02-01T09:51:34Z,39c5444fafca78435ae4defe63230bbe6af76599,https://api.github.com/repos/openzipkin/zipkin...,We didn't defensively check if autocomplete ta...,312
569446,zipkin,2376,codefromthecrypt,User,closed,2019-02-01T08:17:38Z,2019-02-01T08:58:12Z,2019-02-01T08:58:10Z,2019-02-01T08:58:10Z,d401811bfc3b00d52ba42fa3fa1c54171fd2f4d7,https://api.github.com/repos/openzipkin/zipkin...,NaN,312


In [79]:
unique_count = pr['project_id'].isna()
print(unique_count)

0         False
1         False
2         False
3         False
4         False
          ...  
569443    False
569444    False
569445    False
569446    False
569447    False
Name: project_id, Length: 569448, dtype: bool


In [85]:
has_nan = pr['project_id'].isna().any()
has_nan

False

In [84]:
pr.to_csv("csvs/modified_pull-requests.csv",index=False)